In [ ]:
!pip3 install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 20.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=bdb0a8cdf2da6401d9241cbeae387e358afa5adf30942beb9ee43b7308d5b289
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("EDA")\
.config("spark.driver.maxResultSize","4g")\
.getOrCreate()

sc = spark.sparkContext

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!unzip /content/drive/MyDrive/Colab\ Notebooks/alfabattle2_train_transactions_contest.zip

Archive:  /content/drive/MyDrive/Colab Notebooks/alfabattle2_train_transactions_contest.zip
 extracting: alfabattle2_train_transactions_contest/.Rhistory  
  inflating: alfabattle2_train_transactions_contest/part_000_0_to_23646.parquet  
  inflating: alfabattle2_train_transactions_contest/part_001_23647_to_47415.parquet  
  inflating: alfabattle2_train_transactions_contest/part_002_47416_to_70092.parquet  
  inflating: alfabattle2_train_transactions_contest/part_003_70093_to_92989.parquet  
  inflating: alfabattle2_train_transactions_contest/part_004_92990_to_115175.parquet  
  inflating: alfabattle2_train_transactions_contest/part_005_115176_to_138067.parquet  
  inflating: alfabattle2_train_transactions_contest/part_006_138068_to_159724.parquet  
  inflating: alfabattle2_train_transactions_contest/part_007_159725_to_180735.parquet  
  inflating: alfabattle2_train_transactions_contest/part_008_180736_to_202834.parquet  
  inflating: alfabattle2_train_transactions_contest/part_009_2028

In [ ]:
final_df = spark.read.parquet('file:///content/alfabattle2_train_transactions_contest/*.parquet')

In [ ]:
final_df.count()

270450065

In [ ]:
df1 = spark.read.format("csv")\
    .option("header","True")\
    .option("inferSchema","True")\
    .load('file:///content/drive/MyDrive/Colab\ Notebooks/alfabattle2_train_target.csv')

In [ ]:
df1.printSchema()

root
 |-- app_id: integer (nullable = true)
 |-- product: integer (nullable = true)
 |-- flag: integer (nullable = true)



In [ ]:
merge_df=final_df.join(df1,"app_id","inner")

In [ ]:
merge_df.count()

270450065

In [ ]:
merge_df.columns

['app_id',
 'amnt',
 'currency',
 'operation_kind',
 'card_type',
 'operation_type',
 'operation_type_group',
 'ecommerce_flag',
 'payment_system',
 'income_flag',
 'mcc',
 'country',
 'city',
 'mcc_category',
 'day_of_week',
 'hour',
 'days_before',
 'weekofyear',
 'hour_diff',
 'transaction_number',
 '__index_level_0__',
 'app_id',
 'product',
 'flag']

In [ ]:
merge_df.select('amnt','hour','days_before','weekofyear','hour_diff').summary().show()

+-------+-------------------+------------------+------------------+------------------+------------------+
|summary|               amnt|              hour|       days_before|        weekofyear|         hour_diff|
+-------+-------------------+------------------+------------------+------------------+------------------+
|  count|          270450065|         270450065|         270450065|         270450065|         270450065|
|   mean|0.35886466007912515|14.107251074241747|163.39620992511132|25.617661230734036|26.647970862939154|
| stddev|0.09892837081582107| 4.872842067439201|102.62945233643887|14.918086830236431| 88.31970841676157|
|    min|                0.0|                 0|                 1|                 1|                -1|
|    25%| 0.2929344833513306|                11|                74|                13|                 1|
|    50%| 0.3479347025901494|                14|               155|                25|                 5|
|    75%| 0.4150715755573277|                1

In [ ]:
for column in merge_df.columns:
    print(column,': ',merge_df.select(column).distinct().count())

app_id :  963811
amnt :  3823337
currency :  11
operation_kind :  7
card_type :  173
operation_type :  22
operation_type_group :  4
ecommerce_flag :  3
payment_system :  7
income_flag :  3
mcc :  108
country :  24
city :  163
mcc_category :  28
day_of_week :  7
hour :  24
days_before :  359
weekofyear :  53
hour_diff :  6345
transaction_number :  9580
__index_level_0__ :  5573539
product :  5
flag :  2


In [ ]:
merge_df.select('currency').distinct().show()

+--------+
|currency|
+--------+
|       1|
|       6|
|       3|
|       5|
|       9|
|       4|
|       8|
|       7|
|      10|
|      11|
|       2|
+--------+



In [ ]:
merge_df.select('operation_kind').distinct().show()

+--------------+
|operation_kind|
+--------------+
|             1|
|             6|
|             3|
|             5|
|             4|
|             7|
|             2|
+--------------+



In [ ]:
merge_df.select('card_type').distinct().count()

173

In [ ]:
merge_df.printSchema()

root
 |-- app_id: integer (nullable = true)
 |-- amnt: double (nullable = true)
 |-- currency: integer (nullable = true)
 |-- operation_kind: integer (nullable = true)
 |-- card_type: integer (nullable = true)
 |-- operation_type: integer (nullable = true)
 |-- operation_type_group: integer (nullable = true)
 |-- ecommerce_flag: integer (nullable = true)
 |-- payment_system: integer (nullable = true)
 |-- income_flag: integer (nullable = true)
 |-- mcc: integer (nullable = true)
 |-- country: integer (nullable = true)
 |-- city: integer (nullable = true)
 |-- mcc_category: integer (nullable = true)
 |-- day_of_week: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- days_before: integer (nullable = true)
 |-- weekofyear: integer (nullable = true)
 |-- hour_diff: long (nullable = true)
 |-- transaction_number: integer (nullable = true)
 |-- __index_level_0__: long (nullable = true)
 |-- app_id: integer (nullable = true)
 |-- product: integer (nullable = true)
 |-- flag:

In [ ]:
merge_df.select('operation_type').distinct().count()

22

In [ ]:
merge_df.select('payment_system').distinct().show()

+--------------+
|payment_system|
+--------------+
|             1|
|             6|
|             3|
|             5|
|             4|
|             7|
|             2|
+--------------+



In [ ]:
merge_df.select('mcc_category').distinct().count()

28